In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import json
from thefuzz import fuzz, process
from tenacity import retry, stop_after_attempt, wait_fixed, Retrying

pd.options.mode.chained_assignment =  None

data_DIR = "../../data/injuries/"
export_DIR = "../../../repos/csv/"
# Pretending to be a browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

with open("../../data/NBA.json") as f:
    data = json.load(f)
pID_dict = {v: int(k) for k, v in data.items()}
player_dict = {int(k): v for k, v in data.items()}

## Since 2000

In [ ]:
# Start Date
start_date = "2000-01-01"

In [ ]:
df0 = pd.DataFrame()
print(start_date)

In [ ]:
#URL to scrape from 
url = f"https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={start_date}&EndDate=&PlayerMovementChkBx=yes&Submit=Search"

In [ ]:
#-------------Scrape web page--------------------------------------

#Get URL HTML
response = requests.get(url)
print(response) # Response [200] means it went through

#Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

#-------------Scrape data from the first web page----------------
#Read in html as pandas data frame
df_first_page = pd.read_html(url,storage_options=header)
    
#Select table of interest (the first table)
df_first_page = df_first_page[0]

#Drop first row (column names)
df_first_page.drop([0], inplace = True)
   
#Remove bullet in front of player names
df_first_page[2]=df_first_page[2].str[2:] # "Acquired" column
df_first_page[3]=df_first_page[3].str[2:] # "Relinquished" column
    
#Modify column titles
df_first_page.columns = ['Date','Team','Acquired','Relinquished','Notes']

dfa = []
#data frame list to hold data for concating later
dfa.append(df_first_page)

In [ ]:
#------------Scrape data from other pages linked at the bottom of the first page------------
# Loop over links (skipping the first 4 (not data) and last 4 ("Next" and other webpage links))
for i in tqdm(range(4,len(soup.findAll('a'))-4)): #'a' tags are for links
   
    #find all links on webpage and select the i-th link
    one_a_tag = soup.findAll('a')[i]
    link = one_a_tag['href']
    
    #Add in the rest of the url
    download_url = 'https://www.prosportstransactions.com/basketball/Search/'+ link
    # print(download_url)
    
    #Read html as pandas data frame
    dfs = pd.read_html(download_url, storage_options=header)
    
    #Select table of interest (the first table)
    df = dfs[0]
    
    #Drop first row (column names)
    df.drop([0], inplace = True)
   
    #Remove bullet in front of names
    df[2]=df[2].str[2:] # "Acquired" column
    df[3]=df[3].str[2:] # "Relinquished" column
    
    #Modify column titles
    df.columns = ['Date','Team','Acquired','Relinquished','Notes']
    #Add a pause to keep web server happy
    time.sleep(0.2)
    dfa.append(df)

In [ ]:
df11 = pd.concat(dfa)
df = df11.copy()
df = df[~df["Notes"].isna()].reset_index(drop=True)
df.to_parquet(data_DIR + "all_trades.parquet")

In [ ]:
df = pd.read_parquet(data_DIR + "all_trades.parquet")

In [ ]:
df["Team"] = df["Team"].str.replace("Clippets","Clippers")
df["Team"] = df["Team"].str.replace("Grizzles","Grizzlies")
df["Team"] = df["Team"].str.replace("Lakerse","Lakers")
df["Notes"] = df["Notes"].str.replace("Clippets","Clippers")
df["Notes"] = df["Notes"].str.replace("Grizzles","Grizzlies")
df["Notes"] = df["Notes"].str.replace("Lakerse","Lakers")
df = df[~(df["Notes"].str.contains("void"))]

In [ ]:
df1 = df[df["Notes"].str.contains("trade")]
df1 = df1[df1["Date"] >= "2010-07-01"].reset_index(drop=True)
df1["tradetype"] = df1["Notes"].str.split(' ').str[0]
df1 = df1[~(df1["tradetype"] == "earlier")].reset_index(drop=True)
df1 = df1.rename(columns={"Team":"Team1"})

In [ ]:
df1_1 = df1[df1["tradetype"] == "trade"].reset_index(drop=True)
df1_1["Team2"] = df1_1["Notes"].str.split(' ').str[-1]
df1_1.insert(2,"Team2",df1_1.pop("Team2"))

In [ ]:
df1_2 = df1[df1["tradetype"] == "3-team"].reset_index(drop=True)
df1_2["Team2"] = df1_2["Notes"].str.split(' ').str[-2]
df1_2["Team3"] = df1_2["Notes"].str.split(' ').str[-1]
df1_2.insert(2,"Team2",df1_2.pop("Team2"))
df1_2.insert(3,"Team3",df1_2.pop("Team3"))
df1_2["Team2"] = df1_2["Team2"].str.replace(",","")
df1_21 = df1_2.drop(columns= "Team3")
df1_22 = df1_2.drop(columns= "Team2")
df1_22 = df1_22.rename(columns={"Team3":"Team2"})
df1_2 = pd.concat([df1_21,df1_22]).sort_values(["Date","Team1"]).reset_index(drop=True)

In [ ]:
df1[(df1["tradetype"] == "3-team") & (df1["Date"] == "2019-02-07")]

In [ ]:
df1_2[df1_2["Date"] == "2019-02-07"]

In [ ]:
df1_3 = df1[df1["tradetype"] == "4-team"].reset_index(drop=True)
df1_3["Team2"] = df1_3["Notes"].str.split(' ').str[-3]
df1_3["Team3"] = df1_3["Notes"].str.split(' ').str[-2]
df1_3["Team4"] = df1_3["Notes"].str.split(' ').str[-1]
df1_3.insert(2,"Team2",df1_3.pop("Team2"))
df1_3.insert(3,"Team3",df1_3.pop("Team3"))
df1_3.insert(4,"Team4",df1_3.pop("Team4"))
df1_3["Team2"] = df1_3["Team2"].str.replace(",","")
df1_3["Team3"] = df1_3["Team3"].str.replace(",","")
df1_31 = df1_3.drop(columns= ["Team3","Team4"])
df1_32 = df1_3.drop(columns= ["Team2","Team4"])
df1_33 = df1_3.drop(columns= ["Team2","Team3"])
df1_32 = df1_32.rename(columns={"Team3":"Team2"})
df1_33 = df1_33.rename(columns={"Team4":"Team2"})
df1_3 = pd.concat([df1_31,df1_32,df1_33]).sort_values(["Date","Team1"]).reset_index(drop=True)

df1_4 = df1[df1["tradetype"] == "5-team"].reset_index(drop=True)
df1_4["Team2"] = df1_4["Notes"].str.split(' ').str[-4]
df1_4["Team3"] = df1_4["Notes"].str.split(' ').str[-3]
df1_4["Team4"] = df1_4["Notes"].str.split(' ').str[-2]
df1_4["Team5"] = df1_4["Notes"].str.split(' ').str[-1]
df1_4.insert(2,"Team2",df1_4.pop("Team2"))
df1_4.insert(3,"Team3",df1_4.pop("Team3"))
df1_4.insert(4,"Team4",df1_4.pop("Team4"))
df1_4.insert(5,"Team5",df1_4.pop("Team5"))
df1_4["Team2"] = df1_4["Team2"].str.replace(",","")
df1_4["Team3"] = df1_4["Team3"].str.replace(",","")
df1_4["Team4"] = df1_4["Team4"].str.replace(",","")
df1_41 = df1_4.drop(columns= ["Team3","Team4","Team5"])
df1_42 = df1_4.drop(columns= ["Team2","Team4","Team5"])
df1_43 = df1_4.drop(columns= ["Team2","Team3","Team5"])
df1_44 = df1_4.drop(columns= ["Team2","Team3","Team4"])
df1_42 = df1_42.rename(columns={"Team3":"Team2"})
df1_43 = df1_43.rename(columns={"Team4":"Team2"})
df1_44 = df1_44.rename(columns={"Team5":"Team2"})
df1_4 = pd.concat([df1_41,df1_42,df1_43,df1_44]).sort_values(["Date","Team1"]).reset_index(drop=True)

df2 = pd.concat([df1_1,df1_2,df1_3,df1_4]).sort_values(["Date","Team1"]).reset_index(drop=True)

In [ ]:
"hired as general manager"
"hired as president of basketball operations"
"hired as president & general manager"
"hired as vice president of basketball operations"
"hired as VP of basketball operations"

"promoted to general manager"
"promoted to president of basketball operations"
"promoted to president & general manager"
"promoted to vice president of basketball operations"
"promoted to VP of basketball operations"

In [ ]:
df3_1 = df[df["Notes"].str.contains("hired as general manager")]
df3_2 = df[df["Notes"].str.contains("hired as president of basketball operations")]
df3_3 = df[df["Notes"].str.contains("hired as president & general manager")]
df3_4 = df[df["Notes"].str.contains("hired as vice president of basketball operations")]
df3_5 = df[df["Notes"].str.contains("hired as VP of basketball operations")]
df3_6 = df[df["Notes"].str.contains("hired as executive vice president of basketball operations")]
df3_7 = df[df["Notes"].str.contains("promoted to general manager")]
df3_8 = df[df["Notes"].str.contains("promoted to president of basketball operations")]
df3_9 = df[df["Notes"].str.contains("promoted to president & general manager")]
df3_10 = df[df["Notes"].str.contains("promoted to vice president of basketball operations")]
df3_11 = df[df["Notes"].str.contains("promoted to VP of basketball operations")]
df3_12 = df[df["Notes"].str.contains("promoted to executive vice president of basketball operations")]

In [ ]:
df3 = pd.concat([df3_1,df3_2,df3_3,df3_4,df3_5,df3_6,df3_7,df3_8,df3_9,df3_10,df3_11,df3_12])

In [ ]:
df3 = df3.sort_values(["Team","Date"]).reset_index(drop=True)

In [ ]:
# df3.to_csv("executives.csv",index=False)

In [ ]:
df4_1 = df2.query("Date < '2013-07-01'")
df4_2 = df2.query("Date >= '2013-07-01'")

In [ ]:
df4_1["Team1"] = df4_1["Team1"].str.replace("Hornets","Pelicans")
df4_1["Team2"] = df4_1["Team2"].str.replace("Hornets","Pelicans")
df4_1["Team1"] = df4_1["Team1"].str.replace("Bobcats","Hornets")
df4_1["Team2"] = df4_1["Team2"].str.replace("Bobcats","Hornets")
df4_2["Team1"] = df4_2["Team1"].str.replace("Bobcats","Hornets")
df4_2["Team2"] = df4_2["Team2"].str.replace("Bobcats","Hornets")

In [ ]:
df4 = pd.concat([df4_1,df4_2]).sort_values(["Date","Team1"]).reset_index(drop=True)

In [ ]:
len(df4["Team1"].unique())

In [ ]:
df3  = pd.read_csv("executives.csv")
df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")

In [ ]:
df3 = df3.drop(columns=["Relinquished","Notes"])
df3 = df3.rename(columns={"Acquired":"Executive"})

In [ ]:
df3

In [ ]:
end_date = dt.datetime(year=2024, month=6,day=30)
end_date = datetime(end_date.year, end_date.month, end_date.day)

In [ ]:
df3g = df3.groupby("Team")
keys = list(df3g.groups)

In [ ]:
dfia = []
for key in keys:
    dfi = df3g.get_group(key)
    dfi = pd.concat([dfi.head(1),dfi])
    dfi["Date1"] = dfi["Date"].shift(periods=-1)
    dfi["Date1"].iloc[-1] = end_date
    dfi['Dater'] = dfi.apply(lambda row: pd.date_range(row["Date"], row['Date1'], freq='D'), axis=1)
    dfj = dfi.explode('Dater').reset_index(drop=True) \
            .drop(columns=['Date', 'Date1']) \
            .rename(columns={'Dater': 'Date'}) 
    dfj.insert(0,"Date",dfj.pop("Date"))
    dfia.append(dfj)
dfe = pd.concat(dfia)

In [ ]:
dfe.head(2)

In [ ]:
df4["Date"] = pd.to_datetime(df4["Date"], format="%Y-%m-%d")
df5_1 = pd.merge(df4,dfe,left_on=["Date","Team1"],right_on=["Date","Team"])
df5_1 = df5_1.rename(columns={"Executive":"Executive1"})
df5_1 = df5_1.drop(columns="Team")
df5_2 = pd.merge(df5_1,dfe,left_on=["Date","Team2"],right_on=["Date","Team"])
df5 = df5_2.rename(columns={"Executive":"Executive2"})
df5 = df5.drop(columns="Team")

In [ ]:
df5.to_csv("../../data/misc/" + "Trades_w_Executives.csv")

## Current Season

In [62]:
def get_missing_pId(player, player_dict):
    pId = process.extract(player, player_dict, scorer=fuzz.partial_ratio, limit=1)[0][2]
    return pId
def get_missing_tId(team, team_dict):
    tId = process.extract(team, team_dict, scorer=fuzz.partial_ratio, limit=1)[0][2]
    return tId

In [63]:
team_data = pd.read_csv("../../data/NBA_teams_database.csv")
team_list = team_data["TeamID"].tolist()
team_dict1 = team_data.to_dict(orient="records")
team_dict = {team["TeamID"]: team["Team"] for team in team_dict1}

In [84]:
# Start Date
start_date = "2024-10-22"
end_date = "2025-04-13"
# start_date = "2023-10-24"
# end_date = "2024-04-14"

In [85]:
#URL to scrape from 
url = f"https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={start_date}&EndDate={end_date}&PlayerMovementChkBx=yes&Submit=Search"

#-------------Scrape web page--------------------------------------

#Get URL HTML
response = requests.get(url)
print(response) # Response [200] means it went through

#Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

#-------------Scrape data from the first web page----------------
#Read in html as pandas data frame
df_first_page = pd.read_html(url,storage_options=header)
    
#Select table of interest (the first table)
df_first_page = df_first_page[0]

#Drop first row (column names)
df_first_page.drop([0], inplace = True)
   
#Remove bullet in front of player names
df_first_page[2]=df_first_page[2].str[2:] # "Acquired" column
df_first_page[3]=df_first_page[3].str[2:] # "Relinquished" column
    
#Modify column titles
df_first_page.columns = ['Date','Team','Acquired','Relinquished','Notes']

dfa = []
#data frame list to hold data for concating later
dfa.append(df_first_page)

#------------Scrape data from other pages linked at the bottom of the first page------------
# Loop over links (skipping the first 4 (not data) and last 4 ("Next" and other webpage links))
for i in tqdm(range(4,len(soup.findAll('a'))-4)): #'a' tags are for links
   
    #find all links on webpage and select the i-th link
    one_a_tag = soup.findAll('a')[i]
    link = one_a_tag['href']
    
    #Add in the rest of the url
    download_url = 'https://www.prosportstransactions.com/basketball/Search/'+ link
    # print(download_url)
    
    #Read html as pandas data frame
    dfs = pd.read_html(download_url, storage_options=header)
    
    #Select table of interest (the first table)
    df = dfs[0]
    
    #Drop first row (column names)
    df.drop([0], inplace = True)
   
    #Remove bullet in front of names
    df[2]=df[2].str[2:] # "Acquired" column
    df[3]=df[3].str[2:] # "Relinquished" column
    
    #Modify column titles
    df.columns = ['Date','Team','Acquired','Relinquished','Notes']
    #Add a pause to keep web server happy
    time.sleep(0.2)
    dfa.append(df)
df11 = pd.concat(dfa)
df = df11.copy()
df = df[~df["Notes"].isna()].reset_index(drop=True)

<Response [200]>


100%|██████████| 3/3 [00:02<00:00,  1.17it/s]


In [86]:
df1 = df[df["Notes"].str.contains("trade")]
df1["In"]=df1['Acquired'].str.split("•")
df1["Out"]=df1['Relinquished'].str.split("•")
df21 = df1.explode("In")
df21 = df21[["Date","Team","In"]]
df21.columns = ["Date","Team","Player"]
df21["In"] = True
df21["Out"] = False
df22 = df1.explode("Out")
df22 = df22[["Date","Team","Out"]]
df22.columns = ["Date","Team","Player"]
df22["In"] = False
df22["Out"] = True
df2 = pd.concat([df21,df22])
df2 = df2[~df2["Player"].str.contains("pick")]
df2 = df2[~df2["Player"].str.contains("cash")]
df2 = df2[~df2["Player"].str.contains("rights")]
df2 = df2.sort_values(by=["Date","Player","In"]).reset_index(drop=True)
df2["Player"].loc[df2["Player"].str.contains("Kenyon Martin Jr")] = "Kenyon Martin Jr."
df2["playerID"] = df2["Player"].map(pID_dict)

df2.loc[df2["playerID"].isna(),"playerID"] = df2.loc[df2["playerID"].isna(),"Player"].apply(lambda x: get_missing_pId(x, player_dict))
df2["playerID"] = df2["playerID"].astype(int)
df2["teamID"] = df2["Team"].apply(lambda x: get_missing_tId(x, team_dict))
df2.to_parquet(data_DIR + "trades_in_season_2024.parquet")

In [87]:
df2

,Date,Team,Player,In,Out,playerID,teamID
0,2024-12-15,Warriors,Reece Beekman,False,True,1641736,1610612744
1,2024-12-15,Nets,Reece Beekman,True,False,1641736,1610612751
2,2024-12-15,Warriors,De'Anthony Melton,False,True,1629001,1610612744
3,2024-12-15,Nets,De'Anthony Melton,True,False,1629001,1610612751
4,2024-12-15,Nets,Dennis Schröder,False,True,203471,1610612751
5,2024-12-15,Warriors,Dennis Schröder,True,False,203471,1610612744
6,2024-12-15,Heat,Thomas Bryant,False,True,1628418,1610612748
7,2024-12-15,Pacers,Thomas Bryant,True,False,1628418,1610612754
8,2024-12-29,Lakers,Maxwell Lewis,False,True,1641721,1610612747
9,2024-12-29,Nets,Maxwell Lewis,True,False,1641721,1610612751
